# Electricity Generation from api.electricitymap.org
---

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

In [427]:
# Dependencies and Setup

import pandas as pd
import requests


In [428]:

# zones for electrical utilities in US
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO", "US-CAL-BANC","US-CAL-BANC", "US-CAL-TIDC", 
          "US-CAR-CPLE", "US-CAR-CPLW", "US-CAR-DUK", "US-CAR-SC", "US-CAR-SCEG", "US-CAR-YAD", "US-FLA-FMPP", "US-FLA-FPC" , "US-FLA-FPL",
          "US-FLA-GVL" , "US-FLA-HST", "US-FLA-JEA", "US-FLA-SEC", "US-FLA-TAL", "US-FLA-TEC", "US-MIDW-AECI" , "US-MIDW-LGEE", "US-MIDW-MISO",
          "US-NE-ISNE", "US-NW-BPAT", "US-NW-CHPD", "US-NW-DOPD", "US-NW-GCPD", "US-NW-GRID",  "US-NW-IPCO" , "US-NW-NWMT", "US-NW-NEVP", 
           "US-NW-PACW",  "US-NW-PGE", "US-NW-PSEI", "US-NW-SCL", "US-NW-TPWR", "US-NW-WAUW", "US-NY-NYIS", "US-SE-SEPA", "US-SE-SOCO" , 
           "US-TEN-TVA"]


In [429]:

# get carbon intensity history for the US utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url,headers={"auth-token": f"zheg2KxORQLaW"})
    responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file
import json

file_path = "C_intensity_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(responses_dict, json_file, indent=4)

print("Dictionary successfully exported to JSON file.")

df_carbon_intensity_history = pd.read_json(file_path)


Dictionary successfully exported to JSON file.


In [430]:
df_carbon_intensity_history

,response_1,response_2,response_3,response_4,response_5,response_6,response_7,response_8,response_9,response_10,...,response_47,response_48,response_49,response_50,response_51,response_52,response_53,response_54,response_55,response_56
zone,US-SW-PNM,US-SW-EPE,US-SW-WALC,US-NW-PACE,US-NW-PSCO,US-CENT-SWPP,US-TEX-ERCO,US-MIDW-AECI,US-SW-AZPS,US-SW-AZPS,...,US-NW-PACW,US-NW-PGE,US-NW-PSEI,US-NW-SCL,US-NW-TPWR,US-NW-WAUW,US-NY-NYIS,US-SE-SEPA,US-SE-SOCO,US-TEN-TVA
history,"[{'zone': 'US-SW-PNM', 'carbonIntensity': 166,...","[{'zone': 'US-SW-EPE', 'carbonIntensity': 591,...","[{'zone': 'US-SW-WALC', 'carbonIntensity': 59,...","[{'zone': 'US-NW-PACE', 'carbonIntensity': 587...","[{'zone': 'US-NW-PSCO', 'carbonIntensity': 498...","[{'zone': 'US-CENT-SWPP', 'carbonIntensity': 2...","[{'zone': 'US-TEX-ERCO', 'carbonIntensity': 38...","[{'zone': 'US-MIDW-AECI', 'carbonIntensity': 6...","[{'zone': 'US-SW-AZPS', 'carbonIntensity': 478...","[{'zone': 'US-SW-AZPS', 'carbonIntensity': 478...",...,"[{'zone': 'US-NW-PACW', 'carbonIntensity': 179...","[{'zone': 'US-NW-PGE', 'carbonIntensity': 342,...","[{'zone': 'US-NW-PSEI', 'carbonIntensity': 318...","[{'zone': 'US-NW-SCL', 'carbonIntensity': 24, ...","[{'zone': 'US-NW-TPWR', 'carbonIntensity': 24,...","[{'zone': 'US-NW-WAUW', 'carbonIntensity': 152...","[{'zone': 'US-NY-NYIS', 'carbonIntensity': 281...","[{'zone': 'US-SE-SEPA', 'carbonIntensity': 24,...","[{'zone': 'US-SE-SOCO', 'carbonIntensity': 429...","[{'zone': 'US-TEN-TVA', 'carbonIntensity': 452..."


In [431]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl,headers={"auth-token": f"zheg2KxORQLaW"})
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file

file_path = "power_breakdown_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(power_breakdown_responses_dict, json_file, indent=4)

df_power_breakdown_history = pd.read_json(file_path)

In [432]:
df_power_breakdown_history

,response_1,response_2,response_3,response_4,response_5,response_6,response_7,response_8,response_9,response_10,...,response_47,response_48,response_49,response_50,response_51,response_52,response_53,response_54,response_55,response_56
zone,US-SW-PNM,US-SW-EPE,US-SW-WALC,US-NW-PACE,US-NW-PSCO,US-CENT-SWPP,US-TEX-ERCO,US-MIDW-AECI,US-SW-AZPS,US-SW-AZPS,...,US-NW-PACW,US-NW-PGE,US-NW-PSEI,US-NW-SCL,US-NW-TPWR,US-NW-WAUW,US-NY-NYIS,US-SE-SEPA,US-SE-SOCO,US-TEN-TVA
history,"[{'zone': 'US-SW-PNM', 'datetime': '2025-03-07...","[{'zone': 'US-SW-EPE', 'datetime': '2025-03-07...","[{'zone': 'US-SW-WALC', 'datetime': '2025-03-0...","[{'zone': 'US-NW-PACE', 'datetime': '2025-03-0...","[{'zone': 'US-NW-PSCO', 'datetime': '2025-03-0...","[{'zone': 'US-CENT-SWPP', 'datetime': '2025-03...","[{'zone': 'US-TEX-ERCO', 'datetime': '2025-03-...","[{'zone': 'US-MIDW-AECI', 'datetime': '2025-03...","[{'zone': 'US-SW-AZPS', 'datetime': '2025-03-0...","[{'zone': 'US-SW-AZPS', 'datetime': '2025-03-0...",...,"[{'zone': 'US-NW-PACW', 'datetime': '2025-03-0...","[{'zone': 'US-NW-PGE', 'datetime': '2025-03-07...","[{'zone': 'US-NW-PSEI', 'datetime': '2025-03-0...","[{'zone': 'US-NW-SCL', 'datetime': '2025-03-07...","[{'zone': 'US-NW-TPWR', 'datetime': '2025-03-0...","[{'zone': 'US-NW-WAUW', 'datetime': '2025-03-0...","[{'zone': 'US-NY-NYIS', 'datetime': '2025-03-0...","[{'zone': 'US-SE-SEPA', 'datetime': '2025-03-0...","[{'zone': 'US-SE-SOCO', 'datetime': '2025-03-0...","[{'zone': 'US-TEN-TVA', 'datetime': '2025-03-0..."


SECTION 2     
PowerBreakdown data transformation

In [433]:
# pull data from power breakdown json in dataframe
region = df_power_breakdown_history['response_1']['history'][0]['zone']
datetime = df_power_breakdown_history['response_1']['history'][0]['datetime']
nuclear = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = df_power_breakdown_history['response_1']['history'][0]["renewablePercentage"]
total_consumption = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionTotal"]
estimated = df_power_breakdown_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1 = {'region':region,'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}

# Create a dataFrame with the first values
df_US = pd.DataFrame.from_dict(us_pnm1,orient='index')


In [434]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_power_breakdown_history[f"{response}"]['history'][i]['zone']
        datetime = df_power_breakdown_history[f"{response}"]['history'][i]['datetime']
        nuclear = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['nuclear']
        geothermal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['geothermal']
        biomass = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['biomass']
        coal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['coal']
        wind = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['wind']
        solar = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['solar']
        hydro = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro']
        gas = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['gas']
        oil = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['oil']
        unknown = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['unknown']
        hydro_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro discharge']
        battery_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['battery discharge']
        renewable_percentage = df_power_breakdown_history[f"{response}"]['history'][i]["renewablePercentage"]
        total_consumption = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionTotal"]
        estimated = df_power_breakdown_history[f"{response}"]['history'][i]["isEstimated"]

        # this 24 is also for the 23 hours of data for each zone
        df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}
#set up the times as rows and measurements as columns
df_US_new = df_US.transpose()

# check data types
df_US_new.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_9260\3249227210.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar,


,region,datetime,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated
count,1344,1344,1321,1321,1321,1321,1321,1321,1321,1321,1321,1321,1321,1321,1321,1321,1344
unique,53,24,290,20,49,541,434,371,516,830,20,227,19,54,98,1078,2
top,US-CAL-BANC,2025-03-07T23:00:00.000Z,0,0,0,0,0,0,0,0,0,0,0,0,100,1,True
freq,48,56,846,1297,1218,648,728,754,377,198,1203,587,1303,1223,192,17,1189


In [435]:
# fill NA values with zeroes for energy values
df_US_new = df_US_new.fillna({'nuclear': 0,'geothermal': 0,'biomass': 0, 'coal': 0, 'wind': 0, 'solar': 0, 
           'hydro': 0, 'gas': 0, 'oil': 0, 'unknown': 0, 'hydro-discharge':0, 'renewable_percentage':0,
           'battery_discharge':0, 'total_consumption':0})
df_US_new.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000
mean,1042.646577,11.802827,15.471726,1196.444196,930.741815,463.763393,429.446429,1850.094494,2.114583,78.169643,11.070685,35.558036,39.168155,6067.330357
std,2245.386112,88.564454,81.878162,3187.995830,3142.194921,1668.066737,967.320257,3788.774702,10.157302,176.796515,104.189980,309.802771,34.657413,11656.235383
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95.000000,0.000000,0.000000,0.000000,0.000000,6.000000,548.500000
50%,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000,45.500000,485.000000,0.000000,2.000000,0.000000,0.000000,31.000000,1809.500000
75%,412.000000,0.000000,0.000000,948.250000,375.000000,160.500000,398.250000,1370.500000,0.000000,54.000000,0.000000,0.000000,64.250000,5454.750000
max,11748.000000,741.000000,585.000000,23718.000000,22770.000000,13707.000000,7484.000000,28997.000000,59.000000,1105.000000,1264.000000,5487.000000,100.000000,71518.000000


In [436]:
# convert measured Energy values to integers in Giga Watts
#convert_dict = {'hydro': int}
convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
              'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int
               }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_US_new = df_US_new.astype(convert_dict)

#check that data types are changed to int
df_US_new.dtypes

region                  object
datetime                object
nuclear                  int32
geothermal               int32
biomass                  int32
coal                     int32
wind                     int32
solar                    int32
hydro                    int32
gas                      int32
oil                      int32
unknown                  int64
hydro-discharge          int32
battery_discharge        int32
renewable_percentage     int32
total_consumption        int32
estimated               object
dtype: object

In [437]:
# Date Time work

# import datetime dependencies

from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]

# convert date time strings
for i in range(len(df_US_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_US_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)

# add the times and dates to new columns in the data frame
df_US_new['UTC time'] = times
df_US_new['UTC date'] = dates
df_US_new['UTC DateTime'] = DateTime

In [438]:
#set the UTC DateTime as the index
df_US_new_reindex = df_US_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_US_newer = df_US_new.drop('datetime', axis=1)
df_US_newer.head()

,region,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,UTC time,UTC date
UTC DateTime,,,,,,,,,,,,,,,,,,
2025-03-07 23:00:00,US-SW-PNM,0,0,0,152,1292,734,0,293,0,0,0,390,84,2861,True,23:00:00,2025-03-07
2025-03-08 00:00:00,US-SW-PNM,0,0,0,207,1261,227,0,379,0,0,0,390,76,2465,True,00:00:00,2025-03-08
2025-03-08 01:00:00,US-SW-PNM,0,0,0,263,1197,166,0,418,0,0,0,390,72,2434,True,01:00:00,2025-03-08
2025-03-08 02:00:00,US-SW-PNM,0,0,0,266,1189,0,0,438,0,0,0,390,69,2282,True,02:00:00,2025-03-08
2025-03-08 03:00:00,US-SW-PNM,0,0,0,255,1232,0,0,432,0,0,0,390,70,2308,True,03:00:00,2025-03-08


Section 3
Transform carbon intensity data

In [439]:
# pull data from C intensity json in dataframe
region = df_carbon_intensity_history['response_1']['history'][0]['zone']
datetime = df_carbon_intensity_history['response_1']['history'][0]['datetime']
carbon_Intensity = df_carbon_intensity_history['response_1']['history'][0]["carbonIntensity"]
estimated = df_carbon_intensity_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1C = {'region':region,'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# Create a dataFrame with the first values
df_US_C = pd.DataFrame.from_dict(us_pnm1C,orient='index')

In [440]:
# Data wrangling from the response to create a legible dataFrame for carbon intensity history

# outer for loop for regions/responses
for reg in range(len(zones)):
    response = f"response_{reg+1}"
    
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_carbon_intensity_history[f"{response}"]['history'][i]['zone']
        datetime = df_carbon_intensity_history[f"{response}"]['history'][i]['datetime']
        carbon_Intensity = df_carbon_intensity_history['response_1']['history'][i]["carbonIntensity"]
        estimated = df_carbon_intensity_history['response_1']['history'][i]["isEstimated"]
        
        # this 24 is also for the 24 hours of data in each zone
        df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# make the datetime the rows and carbon_intensity a column
df_US_C_new = df_US_C.transpose()

# check data types
df_US_C_new.dtypes

C:\Users\mrkol\AppData\Local\Temp\ipykernel_9260\3332989139.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}


region              object
datetime            object
Carbon_Intensity    object
estimated           object
dtype: object

In [441]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'Carbon_Intensity': int}
 
df_US_C_new = df_US_C_new.astype(convert_dict_C)

#check that the datatype has been changed
df_US_C_new.dtypes

region              object
datetime            object
Carbon_Intensity     int32
estimated           object
dtype: object

In [442]:
# add the times and dates to new columns in the data frame   -    This assumes the data for carbon intensity is pulled at the same time as power breakdown
df_US_C_new['UTC time'] = times
df_US_C_new['UTC date'] = dates
df_US_C_new['UTC DateTime'] = DateTime

#set the UTC DateTime as the index
df_US_C_new_reindex = df_US_C_new.set_index('UTC DateTime', inplace=True)
#drop the datetime column that contains a string
df_US_C_newer = df_US_C_new.drop('datetime', axis=1)

Section 4
Merge dataframes

In [443]:
df_power_and_carbon= pd.merge(df_US_newer, df_US_C_newer,on=['UTC DateTime','region','UTC time','UTC date'])



df_power_and_carbon.rename(columns={'Carbon_Intensity':'Carbon_Intensity(gCO2eq/kWh)','total_consumption':'total_consumption(GW)', 'nuclear':'nuclear(GW)', 
                                    'geothermal':'geothermal(GW)', 'biomass':'biomass(GW)', 'coal':'coal(GW)', 'wind':'wind(GW)', 'solar':'solar(GW)', 
                                    'hydro':'hydro(GW)','gas':'gas(GW)', 'region_x': 'region', 'estimated_x': 'breakdown estimated?','estimated_y':'intensity estimated?'}, inplace=True)




df_power_and_carbon.head()

,region,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),breakdown estimated?,UTC time,UTC date,Carbon_Intensity(gCO2eq/kWh),intensity estimated?
UTC DateTime,,,,,,,,,,,,,,,,,,,,
2025-03-07 23:00:00,US-SW-PNM,0,0,0,152,1292,734,0,293,0,0,0,390,84,2861,True,23:00:00,2025-03-07,166,True
2025-03-08 00:00:00,US-SW-PNM,0,0,0,207,1261,227,0,379,0,0,0,390,76,2465,True,00:00:00,2025-03-08,235,True
2025-03-08 01:00:00,US-SW-PNM,0,0,0,263,1197,166,0,418,0,0,0,390,72,2434,True,01:00:00,2025-03-08,274,True
2025-03-08 02:00:00,US-SW-PNM,0,0,0,266,1189,0,0,438,0,0,0,390,69,2282,True,02:00:00,2025-03-08,296,True
2025-03-08 03:00:00,US-SW-PNM,0,0,0,255,1232,0,0,432,0,0,0,390,70,2308,True,03:00:00,2025-03-08,286,True


In [444]:
# import previous cleaned file into a pandas dataframe
df_us_energy = pd.read_csv('data/allRegions/runningUSenergy_data.csv')
df_us_energy_reindex=df_us_energy.set_index("UTC DateTime")
df_us_energy_reindex.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_9260\1527088667.py:2: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_us_energy = pd.read_csv('data/allRegions/runningUSenergy_data.csv')


,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,148525.000000,148525.000000,148525.000000,148525.000000,148525.000000,148525.000000,148525.000000,148525.000000,148535.000000,148535.000000,148535.000000,148535.000000,148437.000000,148497.000000,148525.000000
mean,1097.131392,13.624844,18.824232,1304.181868,789.313516,466.185349,477.670641,3251.942117,2.474178,82.827556,0.932258,20.319480,33.569009,7526.846785,369.443225
std,2388.135563,98.445972,94.231671,3511.308995,2803.344777,2001.851839,1072.083073,6723.925963,27.975094,231.389111,22.231785,283.632704,35.282620,14654.071294,185.876324
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,184.000000,0.000000,0.000000,0.000000,0.000000,3.000000,606.000000,233.000000
50%,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,52.000000,803.000000,0.000000,3.000000,0.000000,0.000000,19.000000,2037.000000,344.000000
75%,650.000000,0.000000,0.000000,946.000000,204.000000,121.000000,447.000000,2116.000000,0.000000,61.000000,0.000000,0.000000,55.000000,6711.000000,485.000000
max,15375.000000,858.000000,2416.000000,49357.000000,31599.000000,25365.000000,65867.000000,79396.000000,2862.000000,13796.000000,1375.000000,7735.000000,100.000000,128873.000000,1250.000000


In [445]:
df_us_energy_reindex.drop_duplicates(['UTC date', 'UTC time', 'region'],keep='last', inplace=True)
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,148525.000000,148525.000000,148525.000000,148525.000000,148525.000000,148525.000000,148525.000000,148525.000000,148535.000000,148535.000000,148535.000000,148535.000000,148437.000000,148497.000000,148525.000000
mean,1097.131392,13.624844,18.824232,1304.181868,789.313516,466.185349,477.670641,3251.942117,2.474178,82.827556,0.932258,20.319480,33.569009,7526.846785,369.443225
std,2388.135563,98.445972,94.231671,3511.308995,2803.344777,2001.851839,1072.083073,6723.925963,27.975094,231.389111,22.231785,283.632704,35.282620,14654.071294,185.876324
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,184.000000,0.000000,0.000000,0.000000,0.000000,3.000000,606.000000,233.000000
50%,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,52.000000,803.000000,0.000000,3.000000,0.000000,0.000000,19.000000,2037.000000,344.000000
75%,650.000000,0.000000,0.000000,946.000000,204.000000,121.000000,447.000000,2116.000000,0.000000,61.000000,0.000000,0.000000,55.000000,6711.000000,485.000000
max,15375.000000,858.000000,2416.000000,49357.000000,31599.000000,25365.000000,65867.000000,79396.000000,2862.000000,13796.000000,1375.000000,7735.000000,100.000000,128873.000000,1250.000000


In [446]:
# concatentate current data with existing file
df_both = pd.concat([df_us_energy_reindex,df_power_and_carbon])

# drop duplicate rows
df_both.drop_duplicates(['UTC date', 'UTC time', 'region'],keep='first', inplace=True)
df_both.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,148949.000000,148949.000000,148949.000000,148949.000000,148949.000000,148949.000000,148949.000000,148949.000000,148959.000000,148959.000000,148959.000000,148959.000000,148861.000000,148921.000000,148949.000000
mean,1097.101417,13.614808,18.813151,1303.878254,789.284104,468.098557,477.385306,3247.342433,2.473291,82.816352,0.984103,20.292000,33.603771,7523.498439,368.826739
std,2387.827515,98.384021,94.190033,3510.645878,2802.676980,2004.020370,1071.575769,6717.370886,27.940642,231.247062,23.485557,283.246061,35.282107,14648.002066,185.974627
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,184.000000,0.000000,0.000000,0.000000,0.000000,3.000000,605.000000,232.000000
50%,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,52.000000,801.000000,0.000000,3.000000,0.000000,0.000000,19.000000,2036.000000,342.000000
75%,650.000000,0.000000,0.000000,945.000000,204.000000,123.000000,446.000000,2112.000000,0.000000,61.000000,0.000000,0.000000,56.000000,6710.000000,484.000000
max,15375.000000,858.000000,2416.000000,49357.000000,31599.000000,25365.000000,65867.000000,79396.000000,2862.000000,13796.000000,1375.000000,7735.000000,100.000000,128873.000000,1250.000000


In [447]:
df_both.to_csv(f'data/allRegions/runningUSenergy_data.csv')

In [448]:
df_both_cleaned = df_both.loc[df_both['breakdown estimated?']==False,:]
df_both_cleaned.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,36475.000000,36475.000000,36475.000000,36475.000000,36475.000000,36475.000000,36475.000000,36475.000000,36475.000000,36475.000000,36475.000000,36475.000000,36377.000000,36459.000000,36475.000000
mean,1423.838958,49.923482,69.173187,1508.916847,1524.661714,625.960521,809.257930,4511.102673,4.318300,89.200740,0.004441,76.235312,32.294719,10697.323350,398.431008
std,2205.047545,183.632527,170.653313,3606.054827,3910.821728,2655.989098,1314.234915,6581.434416,35.425789,222.395694,0.168801,555.145882,32.208846,14781.148074,173.186757
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,267.000000,0.000000,0.000000,0.000000,0.000000,4.000000,842.000000,267.000000
50%,5.000000,0.000000,0.000000,28.000000,44.000000,0.000000,189.000000,1457.000000,0.000000,3.000000,0.000000,0.000000,22.000000,3318.000000,387.000000
75%,2244.000000,0.000000,3.000000,876.000000,682.000000,59.000000,1099.000000,7290.500000,0.000000,62.000000,0.000000,0.000000,52.000000,16084.000000,505.000000
max,14865.000000,858.000000,2416.000000,35359.000000,28363.000000,24666.000000,65867.000000,79396.000000,2054.000000,13796.000000,21.000000,7735.000000,100.000000,117668.000000,1250.000000


In [449]:
df_both_cleaned.to_csv(f'data/allRegions/runningUSenergy_data_filtered.csv')